KMP 的关键是对模式串预处理，计算出一个 前缀函数数组（prefix array），也叫失效函数（failure function）。该数组的每个元素 

prefix[i] 表示：模式串中前 i+1 个字符组成的子串中，最长的「相等前缀」和「相等后缀」的长度。

前缀：从第一个字符开始的连续子串（不包含最后一个字符）。

后缀：以最后一个字符结束的连续子串（不包含第一个字符）。

示例：模式串 ABABC
计算其前缀函数 prefix 数组（索引从 0 开始）：

prefix[0]（子串 A）：无前后缀，值为 0。

prefix[1]（子串 AB）：前缀 A vs 后缀 B，不等，值为 0。

prefix[2]（子串 ABA）：前缀 A vs 后缀 A，相等，长度 1，值为 1。

prefix[3]（子串 ABAB）：前缀 AB vs 后缀 AB，相等，长度 2，值为 2。

prefix[4]（子串 ABABC）：前缀 A/AB/ABA 与后缀 C/BC/ABC 均不等，值为 0。

因此 prefix = [0, 0, 1, 2, 0]。


初始化 prefix[0] = 0（单个字符无前后缀）。

设 j = 0（用于记录当前最长前缀的长度），遍历模式串从索引 i = 1 开始：

若 pattern[i] == pattern[j]：j += 1，prefix[i] = j，继续下一个 i。

若不相等：
若 j > 0，则令 j = prefix[j-1]（回溯到上一个可能的最长前缀），重复比较。

若 j == 0，则 prefix[i] = 0，继续下一个 i。

prefix[k]的概念是 子串【0.。。k】的最长的前缀后缀长度j 所以当找不到下一个对应的数值 改用j-1 用子串【0.。。j-1】最长的前后缀长度 此时大概率回退到这个地方是最省事的 那就是j =prefix[j-1]

In [ ]:
def build_lps(pat:str) -> list[int]:
    m = len(pat)
    lps = [0]*m
    j = 0 
    for i in range(1,m):
        while j and pat[i] != pat[j]:
            j = lps[j-1]
        if pat[i] == pat[j]:
            j += 1
        lps[i] = j
    return lps

def kmp_search(text:str,pat:str) -> int:
    if not pat: return 0
    lps = build_lps(pat)
    j = 0
    for i,ch in enumerate(text):
        while j and ch != pat[j]:
            j = lps[j-1]
        if ch == pat[j]:
            j += 1
            if j == len(pat):
                return i-j+1
    return -1        k

第二部分 前缀和
当我们算出当前位置的前缀和是 s，如果我们之前见过一个前缀和是 s - k，
那就说明：

中间某一段的区间和 = k！

一、核心思想
对于一个数组 nums，定义其前缀和数组 prefix，其中 prefix[i] 表示「数组 nums 中前 i 个元素的和」（或从第 0 个到第 i-1 个元素的和，具体取决于定义方式）。
通过前缀和数组，可以快速推导任意子数组 nums[left...right]（包含 left 和 right）的和：sum(nums[left...right]) = prefix[right+1] - prefix[left]
二、两种常见定义方式

例题1 统计一个整数数组里面 所有和为k的连续子数组的数量

给定nums整数数组 和一个整数k
例如：
若 nums = [1, 1, 1]，k = 2，答案是 2（子数组 [1,1] 有两处：下标 [0,1] 和 [1,2]）。
若 nums = [1, 2, 3]，k = 3，答案是 2（子数组 [1,2] 和 [3]）。

重点思路在于 把s[i]-s[j]的差值等于k的思路 改编成 s[i]-k = s[j]
所以弄一个字典专门记录前缀和 所以当s-k可以在字典里找到 就代表 差值为k

In [ ]:
def subarraySum(nums:List[int],k:int) -> int:
    s = 0
    ans = 0
    prefix_cnt = {0:1}
    for x in nums:
        s += x
        if s-k in prefix_cnt:
            ans+=prefix_cnt[s-k]
        prefix_cnt[s] = prefix_cnt.get(s,0) + 1 
    return ans


关于prefix_cnt.get(s,0)+1
语法
get是字典内置方法 根据键s获取对应的数值 如果键s不在字典中 就返回0
 如果存在 就返回原来的数值 
 +1 表示 再原来的次数上加一
 将计算出的新次数 赋值给字典中间对应的数值 
 prefix_cnt(s) = prefix_cnt.get(s,0)+1